# Neural Networks for Iterative Matrix Algorithms

This notebook demonstrates how neural networks can be designed to perform iterative versions of essential matrix algorithms, specifically:

- Matrix Inversion using iterative gradient descent
- Principal Component Analysis (PCA) using Oja's rule
- Comparison with standard numerical methods

These neural network approaches provide biologically-plausible and online learning alternatives to traditional batch matrix algorithms.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import eigh
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)


## 1. Neural Network for Matrix Inversion

We implement an iterative neural network approach to compute the inverse of a matrix A. The network uses the update rule: W(t+1) = W(t) + η(I - AW(t))W(t)^T, where W converges to A^(-1).


In [ ]:
class NeuralMatrixInversion:
    def __init__(self, learning_rate=0.01):
        self.learning_rate = learning_rate
        self.history = []
    
    def fit(self, A, n_iterations=1000, tol=1e-6):
        """
        Iteratively compute the inverse of matrix A
        """
        n = A.shape[0]
        # Initialize W with small random values
        W = np.random.randn(n, n) * 0.01
        I = np.eye(n)
        
        for i in range(n_iterations):
            # Compute error: I - AW
            error = I - A @ W
            
            # Update rule: W = W + η * error * W^T
            W = W + self.learning_rate * (error @ W + W @ error.T)
            
            # Track convergence
            error_norm = np.linalg.norm(error)
            self.history.append(error_norm)
            
            if error_norm < tol:
                print(f"Converged at iteration {i}")
                break
        
        return W


## 2. Test Matrix Inversion

Let's test our neural network approach on a random symmetric positive definite matrix and compare with NumPy's built-in inverse.


In [ ]:
# Create a symmetric positive definite matrix
n = 5
A_temp = np.random.randn(n, n)
A = A_temp @ A_temp.T + np.eye(n) * 0.1  # Ensure positive definite

# Neural network inversion
nn_inv = NeuralMatrixInversion(learning_rate=0.05)
W_neural = nn_inv.fit(A, n_iterations=2000)

# Standard inversion
A_inv_true = np.linalg.inv(A)

# Compare results
print("\nNeural Network Inverse (first 3x3 block):")
print(W_neural[:3, :3])
print("\nTrue Inverse (first 3x3 block):")
print(A_inv_true[:3, :3])
print("\nFrobenius norm of difference:", np.linalg.norm(W_neural - A_inv_true))

# Verify: A * A_inv should be identity
product = A @ W_neural
print("\nA * W_neural (should be identity):")
print(product[:3, :3])


## 3. Visualize Convergence

Plot the error norm over iterations to visualize how the neural network converges to the matrix inverse.


In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(nn_inv.history, linewidth=2)
plt.xlabel('Iteration', fontsize=12)
plt.ylabel('Error Norm ||I - AW||', fontsize=12)
plt.title('Convergence of Neural Matrix Inversion', fontsize=14)
plt.yscale('log')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


## 4. Oja's Rule for PCA

Oja's rule is a biologically-inspired neural learning rule that extracts the principal component (first eigenvector) of the data covariance matrix. The update rule is: w(t+1) = w(t) + η(x(x^T w) - (w^T x)^2 w), which converges to the first principal component.


In [ ]:
class OjasPCA:
    def __init__(self, n_components=1, learning_rate=0.01):
        self.n_components = n_components
        self.learning_rate = learning_rate
        self.components_ = None
        self.history = []
    
    def fit(self, X, n_iterations=1000):
        """
        Extract principal components using Oja's rule
        X: data matrix (n_samples x n_features)
        """
        n_samples, n_features = X.shape
        
        # Center the data
        X_centered = X - np.mean(X, axis=0)
        
        # Initialize weight vectors
        W = np.random.randn(n_features, self.n_components) * 0.01
        
        for iteration in range(n_iterations):
            # Randomly sample a data point
            idx = np.random.randint(0, n_samples)
            x = X_centered[idx].reshape(-1, 1)
            
            # Oja's rule for multiple components
            y = W.T @ x  # Output
            
            # Update: W = W + η(xy^T - yy^T W)
            dW = self.learning_rate * (x @ y.T - W @ (y @ y.T))
            W = W + dW
            
            # Normalize to prevent unbounded growth
            for j in range(self.n_components):
                W[:, j] = W[:, j] / (np.linalg.norm(W[:, j]) + 1e-8)
            
            # Track convergence every 100 iterations
            if iteration % 100 == 0:
                self.history.append(np.linalg.norm(dW))
        
        self.components_ = W.T
        return self
    
    def transform(self, X):
        """Project data onto principal components"""
        X_centered = X - np.mean(X, axis=0)
        return X_centered @ self.components_.T


## 5. Generate Test Data for PCA

Create synthetic data with known principal components to test Oja's rule.


In [ ]:
# Generate synthetic data with clear principal components
n_samples = 500
n_features = 3

# Create data with variance along specific directions
t = np.linspace(0, 4*np.pi, n_samples)
X_data = np.zeros((n_samples, n_features))
X_data[:, 0] = 3 * np.cos(t) + np.random.randn(n_samples) * 0.1
X_data[:, 1] = 2 * np.sin(t) + np.random.randn(n_samples) * 0.1
X_data[:, 2] = 0.5 * np.random.randn(n_samples)

# Rotate the data
rotation_matrix = np.array([[0.8, -0.6, 0], [0.6, 0.8, 0], [0, 0, 1]])
X_data = X_data @ rotation_matrix.T

print(f"Data shape: {X_data.shape}")
print(f"Data mean: {np.mean(X_data, axis=0)}")
print(f"Data std: {np.std(X_data, axis=0)}")


## 6. Apply Oja's Rule and Compare with Standard PCA

Train the neural network using Oja's rule and compare the extracted principal components with those from standard eigenvalue decomposition.


In [ ]:
# Apply Oja's rule
oja_pca = OjasPCA(n_components=2, learning_rate=0.001)
oja_pca.fit(X_data, n_iterations=50000)

# Standard PCA using eigenvalue decomposition
X_centered = X_data - np.mean(X_data, axis=0)
cov_matrix = (X_centered.T @ X_centered) / (n_samples - 1)
eigenvalues, eigenvectors = eigh(cov_matrix)

# Sort by eigenvalues (descending)
idx = eigenvalues.argsort()[::-1]
eigenvalues = eigenvalues[idx]
eigenvectors = eigenvectors[:, idx]

print("\nOja's Rule - First Principal Component:")
print(oja_pca.components_[0])
print("\nStandard PCA - First Principal Component:")
print(eigenvectors[:, 0])

print("\nOja's Rule - Second Principal Component:")
print(oja_pca.components_[1])
print("\nStandard PCA - Second Principal Component:")
print(eigenvectors[:, 1])

# Compute alignment (absolute dot product, since direction can be flipped)
alignment_1 = abs(np.dot(oja_pca.components_[0], eigenvectors[:, 0]))
alignment_2 = abs(np.dot(oja_pca.components_[1], eigenvectors[:, 1]))
print(f"\nAlignment of 1st component: {alignment_1:.4f} (1.0 = perfect)")
print(f"Alignment of 2nd component: {alignment_2:.4f} (1.0 = perfect)")

print(f"\nEigenvalues (variance explained): {eigenvalues}")


## 7. Visualize PCA Results

Project the data onto the principal components found by both methods and visualize the results.


In [ ]:
# Project data onto principal components
X_oja = oja_pca.transform(X_data)
X_standard = X_centered @ eigenvectors[:, :2]

# Create visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Oja's rule projection
axes[0].scatter(X_oja[:, 0], X_oja[:, 1], alpha=0.5, s=20)
axes[0].set_xlabel('First Principal Component', fontsize=11)
axes[0].set_ylabel('Second Principal Component', fontsize=11)
axes[0].set_title("Oja's Rule PCA Projection", fontsize=13)
axes[0].grid(True, alpha=0.3)
axes[0].axis('equal')

# Standard PCA projection
axes[1].scatter(X_standard[:, 0], X_standard[:, 1], alpha=0.5, s=20, color='orange')
axes[1].set_xlabel('First Principal Component', fontsize=11)
axes[1].set_ylabel('Second Principal Component', fontsize=11)
axes[1].set_title('Standard PCA Projection', fontsize=13)
axes[1].grid(True, alpha=0.3)
axes[1].axis('equal')

plt.tight_layout()
plt.show()


## 8. Visualize Learning Dynamics

Plot the convergence of Oja's rule over training iterations.


In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(oja_pca.history, linewidth=2, color='green')
plt.xlabel('Iteration (x100)', fontsize=12)
plt.ylabel('Weight Update Norm', fontsize=12)
plt.title("Convergence of Oja's Rule", fontsize=14)
plt.yscale('log')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


## 9. Performance Comparison

Compare the computational characteristics of neural network approaches versus standard methods.


In [ ]:
import time

# Time comparison for matrix inversion
sizes = [5, 10, 20, 30]
neural_times = []
standard_times = []

for size in sizes:
    # Create test matrix
    A_temp = np.random.randn(size, size)
    A_test = A_temp @ A_temp.T + np.eye(size) * 0.1
    
    # Neural network approach
    start = time.time()
    nn_inv_test = NeuralMatrixInversion(learning_rate=0.05)
    nn_inv_test.fit(A_test, n_iterations=1000)
    neural_times.append(time.time() - start)
    
    # Standard approach
    start = time.time()
    _ = np.linalg.inv(A_test)
    standard_times.append(time.time() - start)

# Visualization
plt.figure(figsize=(10, 5))
plt.plot(sizes, neural_times, 'o-', label='Neural Network (1000 iter)', linewidth=2, markersize=8)
plt.plot(sizes, standard_times, 's-', label='Standard np.linalg.inv', linewidth=2, markersize=8)
plt.xlabel('Matrix Size', fontsize=12)
plt.ylabel('Time (seconds)', fontsize=12)
plt.title('Computation Time Comparison: Matrix Inversion', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\nKey Observations:")
print("- Standard methods are faster for batch processing")
print("- Neural approaches enable online/incremental learning")
print("- Neural methods are biologically plausible and parallelizable")


## Conclusion

This notebook demonstrated how neural networks can perform iterative versions of essential matrix algorithms:

1. Matrix Inversion: We implemented a neural network that iteratively converges to the matrix inverse using gradient-based updates. The method successfully approximated the inverse with high accuracy.
2. PCA via Oja's Rule: We implemented Oja's biologically-inspired learning rule that extracts principal components through online learning. The extracted components closely matched those from standard eigenvalue decomposition.
3. Trade-offs: While standard numerical methods are faster for batch processing, neural approaches offer advantages in online learning, biological plausibility, and potential for hardware parallelization.

These neural network approaches are particularly valuable in scenarios requiring incremental learning, real-time adaptation, or neuromorphic computing implementations.
